In [1]:
import os

In [2]:
baseDir = os.getcwd()
dataLocation = 'data/short_clean_closepx.csv'
dataFilePath = os.path.join(baseDir, dataLocation)

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
masterData = pd.read_csv(dataFilePath, index_col=0)
masterData.head()

,MSFT,AAPL,AMZN,JPM,GOOG,GOOGL,XOM
Date,,,,,,,
2005-01-03,26.740000,4.520714,44.520000,39.150002,100.700043,101.456459,50.090000
2005-01-04,26.840000,4.567143,42.139999,38.410000,96.621567,97.347351,49.750000
2005-01-05,26.780001,4.607143,41.770000,38.490002,96.129768,96.851852,49.490002
2005-01-06,26.750000,4.610714,41.049999,38.709999,93.665794,94.369370,50.119999
2005-01-07,26.670000,4.946429,42.320000,38.400002,96.298668,97.022018,49.790001


In [ ]:
"""
Add the analysis of how often a stock moves more than 1 std dev.
"""

<h2>Lets look at the rolling correlation</h2>

<ol><li><h4>First reshape the data to a dictionary. Key:stock, Value:columns=years</h4></li></ol>

In [6]:
import my_helpers

In [8]:
byStockAndYear = my_helpers.ByStockAndYear(masterData)

# drop 2019 data
for _,data in byStockAndYear.items():
    del data[2019]

print(byStockAndYear.keys())
print(byStockAndYear['MSFT'].columns) 

dict_keys(['MSFT', 'AAPL', 'AMZN', 'JPM', 'GOOG', 'GOOGL', 'XOM'])
Int64Index([2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
            2016, 2017, 2018],
           dtype='int64')


<ol start="2"><li><h4>Create Rolling Correlation Data</h4></li><ol>

In [9]:
rollCorrByStockAndYear = {}
for stock in byStockAndYear:
    rollCorrByStockAndYear[stock] = byStockAndYear[stock].rolling(window=20)\
                                    .corr().dropna()
print(rollCorrByStockAndYear['MSFT'].head())

             2005      2006      2007      2008      2009      2010      2011  \
20 2005  1.000000  0.292330 -0.608512  0.815974  0.790996  0.510769 -0.081692   
   2006  0.292330  1.000000  0.028904 -0.008812 -0.077044 -0.338333 -0.337276   
   2007 -0.608512  0.028904  1.000000 -0.527576 -0.483272 -0.188820  0.119591   
   2008  0.815974 -0.008812 -0.527576  1.000000  0.843301  0.758033 -0.043737   
   2009  0.790996 -0.077044 -0.483272  0.843301  1.000000  0.842707  0.058714   

             2012      2013      2014      2015      2016      2017      2018  
20 2005 -0.816265 -0.553592 -0.092724  0.187873  0.432200 -0.358442 -0.736829  
   2006 -0.025094  0.210170  0.139333 -0.713706  0.506081  0.523126  0.242809  
   2007  0.432575  0.201919 -0.137315 -0.148218 -0.489618  0.137117  0.439118  
   2008 -0.859393 -0.509269  0.070140  0.391096  0.299552 -0.569538 -0.866443  
   2009 -0.859893 -0.659325 -0.062302  0.463829  0.102078 -0.689567 -0.891701  


<h4><ol start="3"><li>Lets filter the data and look at days and years where:</li><br>
    <ol><li>Corr > +-0.75 and with at least 8 events occuring</li>
        <li>Corr > +-0.65 and with at least 9 events occuring</li>
    </ol></ol>
</h4>

In [10]:
corrResults_A = my_helpers.SeasonCorrTest(rollCorrByStockAndYear,\
                                         dropNum=8, n=0.75)
corrResults_B = my_helpers.SeasonCorrTest(rollCorrByStockAndYear,\
                                         dropNum=9, n=0.65)

MSFT completed
AAPL completed
AMZN completed
JPM completed
GOOG completed
GOOGL completed
XOM completed
MSFT completed
AAPL completed
AMZN completed
JPM completed
GOOG completed
GOOGL completed
XOM completed


In [14]:
corrResults_A['MSFT'].head()

2005  249  2006   -0.825573
           2007   -0.854497
           2009   -0.802987
           2010   -0.774605
           2013    0.847858
dtype: float64

In [15]:
def JustDaysCorrelated(data):
    '''
    Input: returned item from func: SeasonCorrTest()
    Output: dictionary. Key=stock, value= pd.series of the days
        that had high correlation. High correlation was established in 
        SeasonCorrTest()
    '''
    request = {}
    for stock in data:
        days = []
        for n in range(len(data[stock].index)):
            a, b, c = data[stock].index[n]
            days.append(b)
        days = list(set(days))
        days.sort()
        request[stock] = days
    return request

In [17]:
justTheDays_A = JustDaysCorrelated(corrResults_A)
justTheDays_B = JustDaysCorrelated(corrResults_B)

In [23]:
for stock, value in justTheDays_A.items():
    print(stock, f'Num of Days: {len(value)}')
    #print(value)

for stock, value in justTheDays_B.items():
    print(stock, f'Num of Days: {len(value)}')
    #print(value)

MSFT Num of Days: 12
AAPL Num of Days: 16
AMZN Num of Days: 10
JPM Num of Days: 9
GOOG Num of Days: 11
GOOGL Num of Days: 11
XOM Num of Days: 23
MSFT Num of Days: 29
AAPL Num of Days: 34
AMZN Num of Days: 46
JPM Num of Days: 26
GOOG Num of Days: 35
GOOGL Num of Days: 34
XOM Num of Days: 44


In [57]:
def PxReturnForDay(data, pxData):
    request = {}
    for stock, data in data.items():
        requestValue = []
        for day in data:
            # px at the day at which the high correlation occured
            end = pxData[stock].loc[day]
            # px 20 days prior to end day
            start = pxData[stock].loc[(day-20)]
            pctChange = (end-start) / start
            a, b, c = day, round(pctChange.mean(),2),round(pctChange,2)
            #c = round(pctChange.mean(),2)
            requestValue.append((a,b,c))
        request[stock] = requestValue
    return request

In [68]:
a = PxReturnForDay(justTheDays_A, byStockAndYear)
type(a['MSFT'])
for stock, data in a.items():
    for t in data:
        print(t)

(141, 0.04, 2005    0.03
2006    0.06
2007    0.04
2008   -0.05
2009   -0.01
2010    0.06
2011    0.15
2012   -0.02
2013   -0.05
2014    0.06
2015    0.01
2016    0.14
2017    0.05
2018    0.09
dtype: float64)
(142, 0.03, 2005    0.02
2006    0.07
2007    0.00
2008   -0.10
2009   -0.01
2010    0.08
2011    0.11
2012   -0.04
2013   -0.09
2014    0.07
2015    0.00
2016    0.17
2017    0.07
2018    0.12
dtype: float64)
(201, 0.01, 2005   -0.05
2006    0.05
2007    0.10
2008   -0.04
2009    0.04
2010   -0.00
2011    0.01
2012   -0.05
2013    0.04
2014   -0.08
2015    0.08
2016    0.01
2017    0.04
2018   -0.01
dtype: float64)
(202, 0.01, 2005   -0.02
2006    0.05
2007    0.05
2008   -0.05
2009    0.02
2010    0.03
2011    0.04
2012   -0.06
2013    0.07
2014   -0.06
2015    0.09
2016   -0.00
2017    0.05
2018   -0.04
dtype: float64)
(203, 0.02, 2005   -0.02
2006    0.07
2007    0.05
2008   -0.03
2009    0.03
2010    0.04
2011    0.08
2012   -0.08
2013    0.07
2014   -0.04
2015    0.08
2016 

In [67]:
a['MSFT']

[(141, 0.04, 2005    0.03
  2006    0.06
  2007    0.04
  2008   -0.05
  2009   -0.01
  2010    0.06
  2011    0.15
  2012   -0.02
  2013   -0.05
  2014    0.06
  2015    0.01
  2016    0.14
  2017    0.05
  2018    0.09
  dtype: float64), (142, 0.03, 2005    0.02
  2006    0.07
  2007    0.00
  2008   -0.10
  2009   -0.01
  2010    0.08
  2011    0.11
  2012   -0.04
  2013   -0.09
  2014    0.07
  2015    0.00
  2016    0.17
  2017    0.07
  2018    0.12
  dtype: float64), (201, 0.01, 2005   -0.05
  2006    0.05
  2007    0.10
  2008   -0.04
  2009    0.04
  2010   -0.00
  2011    0.01
  2012   -0.05
  2013    0.04
  2014   -0.08
  2015    0.08
  2016    0.01
  2017    0.04
  2018   -0.01
  dtype: float64), (202, 0.01, 2005   -0.02
  2006    0.05
  2007    0.05
  2008   -0.05
  2009    0.02
  2010    0.03
  2011    0.04
  2012   -0.06
  2013    0.07
  2014   -0.06
  2015    0.09
  2016   -0.00
  2017    0.05
  2018   -0.04
  dtype: float64), (203, 0.02, 2005   -0.02
  2006    0.07
  2

In [ ]:
'''
I am working on placing tags on the tuple data. 
I need to make it a dict with the values: Day, Avg, PctReturn
'''